In [182]:
import pandas as pd

preproc_file = "processed_data.csv"

df = pd.read_csv(preproc_file, sep=';', engine='python')

therapy_applied = df.query("TherapyApply == 1")
therapy_not_applied = df.query("TherapyApply == 0")



In [183]:
therapy_applied.head()

,CaseID,Gender,Age,Outcome,Vacin,TherapyApply,Результат_F,Результат_D,Severity,Meds,Duration
0,376917,ж,65.0,Выписан,Нет,1,135.9,423.90,Крайне тяжелое,0,67
1,390158,м,73.0,Выписан,Нет,1,1024.5,578.65,Тяжелое,0,45
2,390185,ж,57.0,Умер,Нет,1,606.0,349.60,Крайне тяжелое,0,44
3,390093,ж,66.0,Умер,Нет,1,586.9,486.70,Крайне тяжелое,0,35
4,390165,м,56.0,Выписан,Спутник V,1,1013.0,417.30,Крайне тяжелое,0,41


In [184]:
therapy_not_applied.head()

,CaseID,Gender,Age,Outcome,Vacin,TherapyApply,Результат_F,Результат_D,Severity,Meds,Duration
2451,390819,ж,79.0,Выписан,Нет,0,565.25,853.6,Тяжелое,0,54
2452,390409,ж,49.0,Выписан,Нет,0,387.90,285.9,Тяжелое,0,45
2453,390794,ж,65.0,Выписан,Нет,0,606.00,349.6,Тяжелое,0,40
2454,390448,м,44.0,Выписан,Нет,0,1082.00,320.6,Среднетяжелое,0,35
2455,390760,ж,69.0,Выписан,Нет,0,586.90,486.7,Тяжелое,0,29


In [185]:
therapy_applied_men = therapy_applied.query("Gender == 'м'")
therapy_applied_women = therapy_applied.query("Gender == 'ж'")

therapy_not_applied_men = therapy_not_applied.query("Gender == 'м'")
therapy_not_applied_women = therapy_not_applied.query("Gender == 'ж'")



In [186]:
therapy_applied_men.head()

,CaseID,Gender,Age,Outcome,Vacin,TherapyApply,Результат_F,Результат_D,Severity,Meds,Duration
1,390158,м,73.0,Выписан,Нет,1,1024.5,578.65,Тяжелое,0,45
4,390165,м,56.0,Выписан,Спутник V,1,1013.0,417.30,Крайне тяжелое,0,41
5,390089,м,63.0,Умер,Нет,1,1013.0,417.30,Тяжелое,0,23
6,390142,м,56.0,Выписан,Нет,1,1013.0,417.30,Тяжелое,0,40
8,390164,м,42.0,Выписан,Нет,1,1082.0,320.60,Крайне тяжелое,0,26


In [187]:

print(f"Men applied therapy {therapy_applied_men.shape}")
print(f"Women applied therapy {therapy_applied_women.shape}")
print(f"Men not applied therapy {therapy_not_applied_men.shape}")
print(f"Women not applied therapy {therapy_not_applied_women.shape}")


Men applied therapy (1059, 11)
Women applied therapy (1392, 11)
Men not applied therapy (2343, 11)
Women not applied therapy (3704, 11)


In [188]:
therapy_applied_men.loc[therapy_applied_men['Результат_F'] > 3000, 'Результат_F'] = 3000
therapy_applied_women.loc[therapy_applied_women['Результат_F'] > 3000, 'Результат_F'] = 3000
therapy_not_applied_men.loc[therapy_not_applied_men['Результат_F'] > 3000, 'Результат_F'] = 3000
therapy_not_applied_women.loc[therapy_not_applied_women['Результат_F'] > 3000, 'Результат_F'] = 3000

therapy_applied_men.loc[therapy_applied_men['Результат_D'] > 3000, 'Результат_D'] = 3000
therapy_applied_women.loc[therapy_applied_women['Результат_D'] > 3000, 'Результат_D'] = 3000
therapy_not_applied_men.loc[therapy_not_applied_men['Результат_D'] > 3000, 'Результат_D'] = 3000
therapy_not_applied_women.loc[therapy_not_applied_women['Результат_D'] > 3000, 'Результат_D'] = 3000


def generate_pairs(applied, not_applied, vacin = False):
    applied = applied.sort_values("Age")
    not_applied = not_applied.sort_values("Age")
    pairs = []
    used = set()

    eps = 1e-10
    for index, first in applied.iterrows():
        basic_match = (not_applied['Gender'] == first['Gender']) & (abs(not_applied['Age'] - first['Age']) <= 3)
        fd_match = (abs(not_applied['Результат_F'] - first['Результат_F']) / not_applied['Результат_F'].replace(0, eps) <= 0.1) & (abs(not_applied['Результат_D'] - first['Результат_D']) / not_applied['Результат_D'].replace(0, eps) <= 0.1)
        severity_match = (((first['Severity'] == "Крайне тяжелое") | (first['Severity'] == "Тяжелое")) & \
                    ((not_applied['Severity'] == "Крайне тяжелое") | (not_applied['Severity'] == "Тяжелое"))) | \
                    ((first['Severity'] == "Среднетяжелое") & (not_applied['Severity'] == "Среднетяжелое"))
        vacin_match = (~vacin) | ( not_applied['Vacin'] == first['Vacin'] )
        mask = basic_match & fd_match & severity_match & vacin_match
        filtered = not_applied[mask]
        for sIndex, second in filtered.iterrows():
            if sIndex not in used:
                pairs.append([first, second])
                used.add(sIndex)
            
    return pairs
pairs_men_vacin = generate_pairs(therapy_applied_men, therapy_not_applied_men, True)
pairs_women_vacin = generate_pairs(therapy_applied_women, therapy_not_applied_women, True)

pairs_men = generate_pairs(therapy_applied_men, therapy_not_applied_men, False)
pairs_women = generate_pairs(therapy_applied_women, therapy_not_applied_women, False)


In [189]:
def make_df(pairs):
    all_data = []
    for pair in pairs:
        all_data.append(pair[0].to_dict())
        all_data.append(pair[1].to_dict())
    return pd.DataFrame(all_data)
df = make_df(pairs_men)


Без учёта вакцинации

In [190]:
print(len(pairs_men))
print(len(pairs_women))

852
1350


С учётом вакцинации

In [191]:
print(len(pairs_men_vacin))
print(len(pairs_women_vacin))

852
1350
